# Policy-Based Mthods

Reinforcement learning is about learning the optimal policy.<br>
So far we have reveiwed value-based methods; we try to find the value function and the policy is implicitly defined by that value function (like when $\epsilon\text{-gradient}$ is used).<br>
We can find the optimal policy without worrying about a value function at all?

## Pros of Policy-Based Methods

Three arguments to consider:
 * Simplicity
 * Stochastic Policies
 * Continuous Action Space

### Simplicity

| method |_____ value-function* _____ | ____________________ policy ____________________ |
|------|------|------|
|value-based| $\color{red}{\hat{q}(s, a)\approx q_*(s, a)}$ | $\color{orange}{\pi\big(\hat{q}(s, a)\big) \approx \pi^*}$|
| policy-based| ---------- | $\color{orange}{\pi \approx \pi^*}$|

*In value-based methods like Q-Learning we use the value function as an intermediate step.

So, in policy-based methods, we look for a way to estimate the optimal policy without intermediate steps.

Such a policy would look like:

$$
\begin{align}
\text{Deterministic approach;     } &\pi\text{ : } s\rightarrow a\\
\text{Stochastic approach;     } &a~\pi(s, a) = \mathbb{P}[a \mid s]
\end{align}$$


Where the *deterministic approach* is just a mapping and the *stochastic approach* is a conditional probability of each action, given a certain state.

We would then choose an action based on this probability distribution. This is simpler because:
 * we directly get at the problem at hand
 * avoid storage of a bunch of additional data that not always is or may be useful (for example large portions of the state space may have the same value. A policy formulated in this way allows to make a generalization where needed/possible and to focus more on the complicated regions of the state space.

### Stochastic Policies

Policy-based methods -alike value vased methods- can learn true stochastic policies. Like for example: picking numbers from a machine, where the probabilities of each number to be selected depend on some state variables that can be changed. In contrast $\epsilon\text{-greedy}$ action selection adds some randomness - but it's really a hack. The underlying value function can still drive us towards certain actions more than others, like for example when playing rock-paper-scissors. In that game, the optimal policy is to choose an action uniformly at random because any other (deterministic or stochastic) policy with some non-uniformity could be exploited by the opponent.<br>
Stochastic policy also helps in cases in which there are aliased states (2 or more states that are perceived by the agent like identical states but that in reallity are different states).

![enyEvironment](./images/anyEnvironment.png)

The status in brown are identical.  The agent cannot distinguish between them and therefore could choose to go left always when in those states, leading eventually to an oscillating situation. It could maybe come out of that oscillatory situation by chance because of $\epsilon\text{-greedy}$ policy but the learning would be anyways very inefficient and could take long and if we sent a high $\epsilon\text{-greedy}$ with the intention of helping the agent to find faster the optimal policy, it might result in bad action in other states.

In an environment like the one depicted in the image above, it would be best to assign equal probabilities to moving left or right from these aliased states.

A value-based approach tends to learn a deterministic or near deterministic policy. A policy-based approach -in situations like the one of the picture above- can learn the desired stochastic policy.

### Continuous Action Spaces

In environments with discrete action spaces we used value-based methods -with or without function approximator. Here the output consist of a value for each state and the value is choosen by the agent - it chooses the highest value.

But if the action space is continuous, the max operation used by the agents in environments with discrete action spaces becomes an optimization problem like for example trying to find the  global maximum of a continuous function - and that is not a trivial task (especailly if the function is not convex).<br>
Similar issues exist in high dimensional action spaces - lots of possible actions to evaluate. It'd be nice if we could map a given state to an action directly even if the resulting policy is a bit more complex, it would significantly reduce the computation time needed to act $\longrightarrow$ an that is something a policy based method can allows to achieve.

## Policy Function Approximation

In order to cover large or continuous state spaces we extend policy-based methods by using a function to approximate the policy - similar to value function approximation.
<center>
    <h4>Policy Function Approximation</h4><br>
    $a \sim \pi(s, a, \theta) = \mathbb{P}[a \mid s, \theta]$
</center>

How? By parameterizing the policy with theta, where theta is the weights of a NeuralNet or any other model. The goal is then, to *optimize theta* to find *the best policy*.<br>
An approximation function is or could be a linear combination of features:
$$f(s, a, \theta) = X(s, a)^\top\theta$$

But this approximation function does not suite us in this case because it does not produce a probability distribution from which we can sample actions and that is what we want to get - a probability distribution of actions.<br>
So in order to produce such probability distribution, we need to use some transformation. We apply the Softmax function that exponentiates the function approximation (the linear combination results) and then normalize it (divides it by the summatory of exponentials for all actions):<br>
$$a \sim \pi(s, a, \theta) = \frac{e^{f(s, a, \theta)}}{\sum_{a'}e^{f(s, a', \theta)}}$$

With this, we get a set of action probabilities that sum to one. <u>However, this works only for discrete action spaces</u>.

When having an environment with a continuous action space we can use a **Gaussian Policy**. In this *Gaussian Policy*, actions are sampled from a Gaussian distribution where the parameters of the distribution are determined by our features:
<center>
    <h4>Linear Function with Gaussian Policy</h4><br>
    $f(s, a, \theta) = X(s, a)^\top\theta$<br>
    $a \sim \pi(s, a, \theta) = N(\mu, \sigma^2)$<br>
    $\begin{align}
    \text{where; }&\\
    &\mu\text{ (mean)} = f(s,a,\theta)\longrightarrow \text{Simple linear combination of features} \\
    &\sigma^2\text{ (variance)}\longrightarrow \text{fixed or parameterized like }\theta\text{ in the Softmax Policy}
    \end{align}$
</center>

The same trick can be applied to any approximation function that produces some values and turn them into probabilities that represent a stochastic policy.

To be able to distinguish which policy is the best we need an objective function.<br>
An objective function is an objective measure. Such measure indicates how good is the policy. This measure must be a function of the policy parameters.
<center>
    <h4>Objective Function</h4><br>
    $J(\theta) = \mathbb{E}_\pi\big[R(\tau)\big]; \tau = S_0, A_0, R_1, S_1 \ldots$
</center>

This function needs to capture the expected rewards obtained under the policy.<br>
Tau (i.e. $\tau$) is a trajectory, that is, a complete or partial episode. The expected value can be computed by sampling over multiple trajectories.

If the problem we have is an episodic task:<br>
An option is to use the mean of the return from the first time step $G_1$. The <b>Start State Value</b>:
<center>
    <h4>Start State Value</h4><br>
$$J_1(\theta) = \mathbb{E}[G_1] = \mathbb{E}\big[V(S_1)\big]
\begin{align}
\text{Where; }&
& G_1 = R_1 + \gamma R_2 + \gamma^2 R_3 + \gamma^3 R_4 + \ldots
\end{align}$$
</center>

As it can be observed in the equation, the return from the first time step $G_1$ is to be used and this is equivalent to using the value of the starting state.

Nevertheless, for continuous environments we cannot depend on a specific starting state. It would be better (for the objective function) to have a measurement that does not depend on the first state. The average or expected state value is a good candidate for this measurement. The <b>Average State Value</b>:
<center>
    <h4>Average State Value</h4><br>
    $J_\bar{v}(\theta) = \mathbb{E}_\pi\big[V(s)\big]$
</center>

You might think that we prefer a policy that leads to a higher average, but there could be states that repeat too often. So we weight each state value by the probability of its occurrence or <u><i>stationary probability</i></u>.<br>
$$J_\bar{v}(\theta) = \mathbb{E}\big[V(s)\big] = \sum_s d(s)v(s) \\
\begin{align}
\text{where; }& \\
& d(s) = \frac{N(s)}{\sum_{s'}N(s')} \\
\end{align}$$

The <u><i>Stationary Probability</i></u>: Divides the number of occurences of the state by the total number of occurrences of all states.

Another measure is the <b>Average Action Value</b> and this one is related to the Average State Value:
<center>
    <h4>Average Action Values</h4><br>
    $$\begin{align}
    J_\bar{q}(\theta) &= \mathbb{E}_\pi\big[Q(s, a)\big] \\
    &= \sum_s d(s)\sum_a \pi(s, a, \theta)Q(s, a)
    \end{align}$$
</center>

Now, let's remember that the goal of encoding the policy directly, is so that no tracking of state values or action values should be needed. This leads us to a measure that can be directly computed; <u>the average reward at each time step</u>:<br>
<center>
    <h4>Average Reward</h4><br>
    $$\begin{align}
    J_\bar{r}(\theta) & =\mathbb{E}_\pi[r] \\
    & = \sum_s d(s)\sum_a\pi(s, a, \theta)R_s^a
    \end{align}$$
</center>

This four formulations work well for optimizing policies. In fact, seems to be that they work equally good.<br>
![ObjectiveFunctionsSummary](./images/ObjectiveFunctionsSummary.png)

## Stochastic Policy Search

Now that we have an objective function and once we have choosen which one we're going to use, we can try to find a policy that maximizes it.<br>
![surfaceFunctionObjective](./images/surfaceFunctionObjective.png)

The figure represents an objective function of two parameters. the height inidcates the policy's  objective value $J(\theta)$.<br>
This idea extends to objective functions of more than two parameters.<br>

### Hill Climbing

Initially, we don't know anything about this surface, so we need to find the spot where the objective value is at it's maximum. One approach would be to nudge the policy around and see what we find.

Say we have an arbitrary policy: $\pi$<br>
The policy is defined by its $\theta$ parameters: $\theta: 0.1 \mid -0.5$<br>
We know how to evaluate the policy:
 * We apply it to the environment
 * We receive then an objective value
Let's imagine that this obtained value lands somewhere on the objective function surface.
 * We change the $\theta$ parameters a little bit
 * We apply the policy again to the environment
 * We receive a new value that again lands somewhere on the objective function surface.
 
The slight change to the $\theta$ parameters can be achieved by adding some *Gaussian noise* to the parameteres. If the new policy's value is better than our previous values, then we set this policy to be our new best policy and we iterate.<br>
This aproach is known as <u>Hill Climbing</u>.

In this approach, the agent literally walk the objective function surface to find the top of a hill.<br>
If we use Hill Climbing, we can use any policy function even if it is not differentiable or continuous.<br>
You can surely see, this approach is not efficient to find the top of the hill.

### Steepest Ascent Hill Climbing

One improvement to Hill Climbing is to choose a number of neighbor policies at each iteration and then to pick the best of those neighbors.<br>
 * We have an arbitrary policy.
 * We apply it to the environment
 * We receive a value and find out where it lands in the surface
 * We choose a small number of neighboring policies by perturbing the parameters randomly
  * Evaluate each neighbor policy by interacting with the environment to get an idea of the neighborhood
 * We pick the policy (the neighbor) that seems to be the bets of the neighbors
 * Iterate
 
This approach is known as <u>Steepest Ascent Hill Climbing</u>.

**Note:** With these approaches we can still get stuck in a local optima. To try to avoid that, we can use random restarts or *simulated annealing*.

### Simulated Annealing

Simulated Annealing uses a schedule to control how the policy space is explored.<br>
It starts with a large noise parameter (radius) that produces larger neighborhoods and it decreases the radius (noise parameter) as it progresses and as it gets closer to the optimal solution.

### Adaptive Noise Scaling

Another approach is the *Adaptive Noise Scaling*. This approach evolves from the Simulated Annealing approach. It adapts the amount of noise parameter (radius) according to the changes in the observed policy values. If it finds a better policy (it means the agent is ascending the Hill) it reduces the search radius (noise parameter) for generating the next policy. By doing this, the agent reduces the variance of the Gaussian noise we add. The *extension* or evolution from Simulated Annealing comes when the agent oberves a policy that is in fact worse that the best policy it has found in the past. In this case, the agent increases the noise parameter (the search radius) and continue with the exploration from the current best policy.

This makes the stochastic policy search less likey to get stock - especailly in domains with a complicated objective function.

## Policy Gradients

Stochastic policy search may or may not always work as expected.<br>
Stochastic policy search happens to be sensitive to the choice of individual samples and may get stuck in local optimal. Besides this it may take long until it converges.<br>
This approach (i.e. Stochastic Policy) is used when we don't know anything about the objective function nor the policy nor the gradient.

*But* if we know something about the policy, the objective function and the gradient of such objective function (or if it could be concluded), then we can take other approaches which are more efficient.<br>
The gradient always points us into the direction of maximum change. So, if we know about such gradient, we do not need to evaluate randomly policies nor their "neighbors". Since -in a case like this- we know the gradient, we can compute the next set of parameters $\theta$ that will give us a better result.<br>
This idea is the basis of *Policy Gradients* and the approach generally goes as follows:
 * Find the gradient for the current policy parameters
 * Update them in the direction of increasing gradient
 * Iterate
 
Since both the policy and the environments are most likely stochastic, we will make small steps i.e. $\alpha$. But this is not strange to us.<br>
Each policy evaluation might yield unconsistent results. For this reason we might want to evaluate a policy over multiple episodes.

![PolicyGradient](./images/PolicyGradients.png)

But if the objective function is not differentiable (which is possible for complicated underlying policies) then we cannot calculate the gradient. In this cases we estimate the gradient by using *finite differences*.<br>
Given any policy $\pi$ defined by an n-dimensional parameter verctor $\theta$, we find the partial derivative of the objective function with respect to each dimension $k$, by adding a tiny value to that particular dimension and computing the difference between the resulting policy value and the current one. Then collect all the derivatives into a single vector to get the combined derivative. Each of these policy evaluations may require at least one full episode of interaction with the environment and we do one evaluation for every parameter dimension at every iteration.

![FiniteDifferences](./images/FiniteDifferences.png)

This procedure takes a *loooooong* time.

If we have access to the underlying policy function, it is more efficient to try and compute the gradient analytically. Here we would need to compute the gradient of the expected value of some function. Even if it sounds difficult, this is a well studied problem. The reward function in the next picture is referred to as the "score function". This expression can be manipulated to estimate the gradient of the score function more easily.

![Computing Gradients Analytically](./images/ComputingGradientsAnalytically.png)

How to do this manipulation?... to simplify things we will consider some arbitrary score function $f$ that is dependent on $X$:
$$\nabla_\theta\mathbb{E}_x\big[f(x)\big]$$<br>
Values of x are drawn from a probability distribution defined by the parameter $\theta$:
$$x \sim \mathbb{P}[x \mid \theta]$$<br>
We can expand the expectation into an integral or summation of the probability of $x$ given $\theta$ times the score function.
$$\begin{align}
\nabla_\theta\mathbb{E}_x\big[f(x)\big] &= \nabla_\theta\sum_x\mathbb{P}[x \mid \theta]f(x) \\
&= \sum_x\nabla_\theta\mathbb{P}[x\mid\theta]f(x)
\end{align}$$<br>
Now we use something called "Likelihood Ratio Trick":
$$ =\sum_x\mathbb{P}[x\mid\theta]\frac{\nabla_\theta\mathbb{P}[x\mid\theta]}{\mathbb{P}[x\mid\theta]}f(x)$$<br>
Then we replace the resulting fraction by the derivative of the log probability:
$$=\sum_x\mathbb{P}[x\mid\theta]\nabla_\theta\big(\text{log }\mathbb{P}[x\mid\theta]\big)f(x)$$
*This is a well known identity*<br>
Finally we convert the summation back to an expectation:
$$=\mathbb{E}\Big[\nabla_\theta\big(\text{log }\mathbb{P}[x\mid\theta]\big)f(x)\Big]$$<br>
Notice that only the derivative of the log probabilities is computed. The derivative of the score function is not computed because it doesn't depend directly on $\theta$.

The gradient expression looks like this:
$$\begin{align}
\nabla_\theta J(\theta) &= \nabla_\theta\mathbb{E}_\pi\big[R(\tau)\big] \\
&=\mathbb{E}_\pi\Big[\nabla_\theta\big(\text{log }\pi(s, a, \theta)\big)R(\tau)\Big]
\end{align}$$<br>
 * Policy function $\pi$  in place of the probability distribution.
 * Our own score function $R$, which can be the sum of all rewards, discounted rewards or some other value-based function.
 * We compute the score function by interacting with the environment and summing the rewards we get. I.e. $\color{red}{R(\tau)}$
 * If the policy function is implemented using an approximator (NeuralNet), then we simply calculate the log of the outputs probabilites and its derivative. I.e. $\color{red}{\nabla_\theta\big(\text{log }\pi(s, a, \theta)\big)}$
 * We compute the expectation stochastically by taking a bunch of samples.
 
 And now we are able to update policy parameters to improve the objective function iteratively:<br>
 <center>
    The Update Rule
 $$\Delta\theta = \alpha\nabla_\theta\big(\text{log }\pi(s, a, \theta)\big)R(\tau)$$
</center>

## Monte Carlo Policy Gradients

![Algorithm Reinforce](./images/AlgorithmReinforcement.png)

## Constrained Policy Gradients

Attention must be paid to the policy parameters in addition to the abjective function. It is actually needed to put some constraints on the policy parameters, li,ke for example, under which circumstances it is allowed to change them or by how much they can be changed (think of robotic applications in which, withouth these constraints, the mechanical parts can be damaged).<br>
Such constraints can be implemented along in the gradient based algorithm in such a way that the difference between two policies is at most some threshold delta:
<center>
    $$\begin{align}
    J(\theta) = \mathbb{E}_\pi\big[R(\tau)\big] \\
    D\big(\pi(\cdot, \cdot, \theta), \pi(\cdot, \cdot, \theta')\big) \leq \delta
    \end{align}$$
</center>

Another way of introducing a constrain is by introducing a penalty term to the objective function. This penalty term is a multiple of the difference between two policies:
<center>
    <h4>Penalty Term</h4><br>
    $J(\theta) = \mathbb{E}_\pi\big[R(\tau)\big] - \beta D\big(\pi(\cdot, \cdot, \theta), \pi(\cdot, \cdot, \theta')\big)$
</center>

If we optimize for this combined objective function, we will get better policies that are not too different at each step. This will stabilize the learning algorithm. This is also like adding regularization to ML algorithms.

We need also a way to qualify the difference between two policies (regardless of which approach we are working with; adding a constraint or a penalty).<br>
The naive way of computing such difference:
$$D\big(\pi(\cdot, \cdot, \theta), \pi(\cdot, \cdot, \theta')\big) = \parallel\theta - \theta\parallel^2 \longrightarrow \text{Parameter Difference}$$

But if the policies are complex functions of the parameters -which is normally the case- then this parameter difference means nothing, since it doesn't accurately reflect the difference on the policy behaviours, what actions they ultimately produce.

If we think of a policy as a probability distribution over actions, we will get a better distance measure between two policies. This perspective is especially useful when having a continuous action space.<br>
When having continuous action spaces the probability distribution (meaning, the policy) over the range of actions can be different from state to state. If we have continuous states, we can unify these into a single continuous distribution over the combined range of states and actions.<br>
Consider the following example:
<H1>PICTURE HERE</H1>

Each of these policies is a distribution over states and actions. How to compare them?<br>
We can use *Kullback Leibler* or KL divergence, which is a statistical measure.
<center>
    <h4>KL Divergence of P with respect to Q</h4><br>
    $\begin{align}
    D_{KL}(P\parallel q) &= \int_{-\infty}^\infty P(x)\text{log}\frac{P(x)}{q(x)}\partial x \\
    &= \int_{-\infty}^\infty P(x)\big(\text{log}p(x) - \text{log}q(x)\big)\partial x
    \end{align}$
</center>


<H1>PICTURE HERE</H1>

We scale the difference by one of the probability values:
$$P(x)\big(\text{log}p(x) - \text{log}q(x)\big)$$

And sum up over the range of x:
$$\int_{-\infty}^\infty P(x)\big(\text{log}p(x) - \text{log}q(x)\big)\partial x$$

This in blue color is the area of the non-overlaping region between the true log probability distributions, but summed up at each location $x$, using one of the probabilities as a reference.<br>
This means that:
$$D_{KL}(P\parallel q) \neq D_{KL}(q\parallel P) \longrightarrow \text{Assymetric distance measure}$$

We apply this to policy gradient by calculating the KL divergence measured between the current policy and the candidate new policy (i.e. between $\theta$ and $\theta'$ and we use this difference as the constraint part of the objective function.<br>
This term punishes the new polices that change the behaviour too much.<br>
The new polilcy has to produce much more value to be consider a better policy.

In the context of the overall policy gradients algorithm, this helps create a more stable learning process where the policy is not jumping around too much.

Other constraint policy gradients:
 * Trust region policy optimization (TRPO)
 * Proximal policy optimization (PPO)